In [1]:
import numpy as np
import progressbar

In [2]:
def split_to_segments(samples, num_segments):
    num_samples_per_segment = samples.size//num_segments
    new_samples = np.split(samples[:num_segments*num_samples_per_segment], num_segments)
    new_samples[-1] = np.vstack([new_samples[-1], samples[num_segments*num_samples_per_segment:]])
    return new_samples

In [3]:
def test_samples_after_pooling(samples, num_segments):
    if np.all(samples==False) or samples.size == 1:
        return 1
    else:
        segments = split_to_segments(samples, num_segments)
        return np.sum([test_samples_after_pooling(segment, num_segments)
                       for segment in segments]) + 1

In [4]:
def probability_of_sample_pattern(samples, infection_probability):
    num_positives = np.sum(samples)
    num_negatives = samples.size - num_positives
    return np.power(infection_probability, num_negatives) + np.power(1 - num_negatives, num_positives)

In [5]:
def generate_samples(population_size, infection_probability):
    samples = np.asarray([np.random.choice([True, False], 1, p=[infection_probability, 1-infection_probability])
               for i in range(population_size)])
    return samples

In [8]:
def monte_carlo_simulation(population_size, num_iterations, infection_probability, num_segments):
    average_num_tests = 0
    for i in range(num_iterations):
        samples = generate_samples(population_size, infection_probability)
        global_iter = 0
        num_tests = test_samples_after_pooling(samples, num_segments)
        average_num_tests += num_tests/population_size/num_iterations
    return average_num_tests

In [15]:
total_population = 10000
num_segments = 100
infection_probability = 0.01
monte_carlo_simulation(total_population, 10, infection_probability, num_segments)

0.6471